# Assignment-1: Transformations and representations

Personal solutions of Avneesh Mishra


# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `Set Up` for detailed step-by-step instructions about the installation setup.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. 
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 11/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.


# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

# Set Up

We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. All assignments will be python based, hence familiarising yourself with Python is essential.


## Setting up Anaconda environment (Recommended)

1. Install Anaconda or Miniconda from [here](https://docs.conda.io/projects/conda/en/latest/user-guide/install/linux.html) depending on your requirements.
2. Now simply run `conda env create -f environment.yml` in the current folder to create an environment `mr_assignment1` (`environment.yml` can be found in `misc/`).
3. Activate it using `conda activate mr_assignment1`.

## Setting up Virtual environment using venv

You can also set up a virtual environment using venv

1. Run `sudo apt-get install python3-venv` from command line.
2. `python3 -m venv ~/virtual_env/mr_assignment1`. (you can set the environment path to anything)
3. `source ~/virtual_env/mr_assignment1/bin/activate`
4. `pip3 install -r requirements.txt` from the current folder (`requirements.txt` can be found in `misc/`).

In [1]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 1. Getting started with Open3D

Open3D is an open-source library that deals with 3D data, such as point clouds, mesh. We'll be using Open3D frequently as we work with point clouds. Let's start with something simple:

<img src="misc/bunny.jpg" alt="drawing" width="200"/>

1. Read the Stanford Bunny file (in `data/`) given to you and visualise it using Open3D.
2. Convert the mesh to a point cloud and change the colour of points.
3. Set a predefined viewing angle (using Open3D) for visualization and display the axes while plotting.
4. Scale, Transform, and Rotate the rabbit (visualise after each step).
5. Save the point cloud as bunny.pcd.

## Answer 1: **Read and Visualize** Stanford bunny file

Let's start with declaring the file name

In [2]:
# Read the mesh file
file_name = "./data/bunny.ply"



- Read using [read_triangle_mesh](http://www.open3d.org/docs/release/tutorial/geometry/file_io.html#Mesh)
- Visualize using [draw_geometries](http://www.open3d.org/docs/release/tutorial/visualization/visualization.html#Function-draw_geometries)

In [3]:

bunny_mesh = o3d.io.read_triangle_mesh(file_name)
# Visualize the mesh
o3d.visualization.draw_geometries([bunny_mesh], "Bunny Mesh", width=1080, height=720)

This gives the following result

![Bunny mesh file](./results/1/1.jpg)

## Answer 2: **Convert Mesh to Point Cloud** and **change color**

Let's set sampling first

In [4]:
N = 2500


- Sample points from mesh to point cloud using [sample_points_uniformly](http://www.open3d.org/docs/release/tutorial/geometry/mesh.html#Sampling)
- Paint point cloud using [paint_uniform_color](http://www.open3d.org/docs/release/tutorial/geometry/pointcloud.html#Paint-point-cloud)

In [5]:
# Convert to point cloud with N points
bunny_pc = bunny_mesh.sample_points_uniformly(N)
# Show both the mesh and the point cloud version
o3d.visualization.draw_geometries([bunny_mesh, bunny_pc], "Bunny PC and Mesh", width=1080, height=720)
# Paint point cloud
c = [252, 123, 3] # RGB 255 color
bunny_pc.paint_uniform_color([c[0]/255, c[1]/255, c[2]/255])
o3d.visualization.draw_geometries([bunny_pc], width=1080, height=720)

The output for drawing point cloud and mesh is as shown below

![Point cloud and mesh](./results/1/2.1.jpg)

The output after giving a custom color to the pointcloud is shown below

![Colored point cloud](./results/1/2.2.jpg)

## Answer 3: Set predefined **viewing angle** for visualization and display **axes** when plotting

Let's define an axis first

- Create a coordinate frame handle using [create_coordinate_frame](http://www.open3d.org/docs/latest/python_api/open3d.geometry.TriangleMesh.html#open3d.geometry.TriangleMesh.create_coordinate_frame)

In [6]:
cf = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2, origin=[0, 0, 0])

Let's create the function that'll do the visualization for us

- Using Customized visualization as shown [here](http://www.open3d.org/docs/release/tutorial/visualization/customized_visualization.html)
    - Get the [ViewControl](http://www.open3d.org/docs/0.12.0/python_api/open3d.visualization.ViewControl.html) object using [get_view_control](http://www.open3d.org/docs/0.12.0/python_api/open3d.visualization.Visualizer.html#open3d.visualization.Visualizer.get_view_control) function

In [7]:
def custom_draw_geometry(pcds, look_at=[0, 0.1, 0], cam_front=[-1, 0, 0.2], cam_up=[0, 1, 0], \
                         title="Custom View", width=1080, height=720):
    """
    Draws geometry with the specified camera parameters
    
    Parameters:
    - pcds: [geometry]
        A list of geometry objects
    - look_at: (3,1)
        A vector that will be at center of window
    - cam_front: (3,1)
        A vector where the camera front is located
    - cam_up: (3, 1)
        A vector that points upwards to the camera
        for orientation information
    - title: str    default: "Custom View"
        The title of window
    - width: int    default: 1080
        Width of display window
    - height: int    default: 720
        Height of the display window
    """
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name=title, width=width, height=height)
    for pcd in pcds:
        vis.add_geometry(pcd)
    ctr = vis.get_view_control()
    ctr.set_lookat(look_at)
    ctr.set_front(cam_front)
    ctr.set_up(cam_up)
    vis.run()
    vis.destroy_window()

In [8]:
# Display coordinate frame and bunny using this visualization
custom_draw_geometry([bunny_pc, cf])

This gives a result like

![Different view and frame](./results/1/3.jpg)

## Answer 4: **Scale**, **Transform** and **Rotate** the point cloud

Let's create a copy first

In [9]:
bunny_tfpc = copy.deepcopy(bunny_pc)
c = [39, 242, 235]
bunny_tfpc.paint_uniform_color([c[0]/255, c[1]/255, c[2]/255])

PointCloud with 2500 points.



[Transformation](http://www.open3d.org/docs/release/tutorial/geometry/transformation.html) module can be used, it has the following functions
- [scale](http://www.open3d.org/docs/release/python_api/open3d.geometry.TriangleMesh.html#open3d.geometry.TriangleMesh.scale) to scale the point cloud about a point
- [translate](http://www.open3d.org/docs/release/python_api/open3d.geometry.TriangleMesh.html#open3d.geometry.TriangleMesh.translate) to translate (move) the point cloud to another point (the given translation vector is added to every point)
- [rotate](http://www.open3d.org/docs/release/python_api/open3d.geometry.TriangleMesh.html#open3d.geometry.TriangleMesh.rotate) to apply a rotation using a 3x3 rotation matrix
    - It uses quaternion convention $\left[ sin(\frac{\theta}{2}), \hat{w_x} cos(\frac{\theta}{2}), \hat{w_y} cos(\frac{\theta}{2}), \hat{w_z} cos(\frac{\theta}{2}) \right ]$ where $\theta$ is the rotation angle (in radians) and $\left[ \hat{w_x} , \hat{w_y}, \hat{w_z}\right]$ is the unit vector which is the axis of rotation. You can use [get_rotation_matrix_from_quaternion](http://www.open3d.org/docs/release/python_api/open3d.geometry.get_rotation_matrix_from_quaternion.html) to get a 3x3 rotation matrix from it
- [transform](http://www.open3d.org/docs/latest/python_api/open3d.geometry.TriangleMesh.html#open3d.geometry.TriangleMesh.transform) to transform (apply a 4x4 homogeneous transformation matrix)

In [10]:
bunny_tfpc = copy.deepcopy(bunny_pc)
c = [39, 242, 235]
bunny_tfpc.paint_uniform_color([c[0]/255, c[1]/255, c[2]/255])
# Scale it
bunny_tfpc.scale(0.5, center=bunny_tfpc.get_center())
custom_draw_geometry([bunny_tfpc, bunny_pc, cf], title="Scaling")
# Translate it
bunny_tfpc.translate([0, 0, 0.1])
custom_draw_geometry([bunny_tfpc, bunny_pc, cf], title="Translation")
# Rotate it
quat_axang = lambda axis, ang: [np.cos(ang/2), axis[0]*np.sin(ang/2), axis[1]*np.sin(ang/2), axis[2]*np.sin(ang/2)]
rot_mat = o3d.geometry.get_rotation_matrix_from_quaternion(quat_axang([1, 0, 0], np.deg2rad(90)))
bunny_tfpc.rotate(rot_mat, center=bunny_tfpc.get_center())
custom_draw_geometry([bunny_tfpc, bunny_pc, cf], title="Rotation")
# Transform it
tf_mat = np.eye(4)
tf_mat[:3, :3] = o3d.geometry.get_rotation_matrix_from_quaternion(quat_axang([0, 0, 1], np.deg2rad(180)))
tf_mat[:3, 3] = [0, 0.1, 0]
bunny_tfpc.transform(tf_mat)
custom_draw_geometry([bunny_tfpc, bunny_pc, cf], title="Transformation", look_at=[0, 0, 0.1], cam_up=[0, 0, 1], \
                     cam_front=[1, 0.1, 0])

The following are the results

![Scaling](./results/1/4.1.jpg)
![Translation](./results/1/4.2.jpg)
![Rotation](./results/1/4.3.jpg)
![Transformation](./results/1/4.4.jpg)

## Answer 5: **Save** the point cloud to a file

Let's declare the file name to use

In [11]:
bunny_tfpc_fname = "./results/1/bunny.pcd"



- Use [write_point_cloud](http://www.open3d.org/docs/release/python_api/open3d.io.write_point_cloud.html#open3d.io.write_point_cloud) function to write a point cloud to a file

In [12]:

o3d.io.write_point_cloud(bunny_tfpc_fname, bunny_tfpc)
# Read it and see if it got saveed properly
bunny_tfpcr = o3d.io.read_point_cloud(bunny_tfpc_fname)
custom_draw_geometry([bunny_tfpcr, cf], title="Saved Bunny", look_at=[0, 0, 0.1], cam_up=[0, 0, 1], \
                     cam_front=[1, 0.1, 0])

This gives the following output

- A file [bunny.pcd](./results/1/bunny.pcd) having the following point cloud

    ![Saved bunny](./results/1/5.jpg)

# 2. Transformations and representations

## a) Euler angles
1. Write a function that returns a rotation matrix given the angles $\alpha$, $\beta$, and $\gamma$ in radians (X-Y-Z)

2. Solve for angles using ```fsolve from scipy``` for three initializations of your choice and compare.
$$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] 
$$

$$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] 
$$

3. What is a Gimbal lock? 

4. Show an example where a Gimbal lock occurs and visualize the Gimbal lock on the given bunny point cloud. You have to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*


### Answer 1: Get Rotation Matrix (XYZ Rotation)
Assuming rotation is in local frame

Start by importing everything

In [13]:
# Import sympy for all symbolic computation
import sympy as sp

Functions for rotation about **X**, **Y** and **Z** axis (for symbolic angles)

In [14]:
# Rotate about X axis
def RotX(angle, degrees = False):
    """
    Generates a Rotation matrice when the rotation is about X axis
    A general rotation about X (Roll) is given by

                | 1    0    0 |
    RotX(T) =   | 0   cT  -sT |
                | 0   sT   cT |

    Where T is rotation angle in radians

    Parameters:
    - angle: Symbol or float
        The angle of rotation
    - degrees: bool     default: False
        If 'True', then angle parameter is assumed to be in degrees
        else it is by default assumed to be in radians
    
    Returns:
    - rot_mat: sp.Matrix        shape: (3, 3)
        The 3x3 rotation matrix for Roll by angle_rad
    """
    angle_rad = angle if not degrees else sp.rad(angle)
    rot_mat = sp.Matrix([
        [1, 0, 0],
        [0, sp.cos(angle_rad), -sp.sin(angle_rad)],
        [0, sp.sin(angle_rad), sp.cos(angle_rad)],
    ])
    return rot_mat

# Rotate about Y Axis
def RotY(angle, degrees = False):
    """
    Generates a Rotation matrice when the rotation is about Y axis
    A general rotation about Y (Pitch) is given by

                |  cT   0   sT |
    RotY(T) =   |   0   1    0 |
                | -sT   0   cT |

    Where T is rotation angle in radians

    Parameters:
    - angle: Symbol or float
        The angle of rotation
    - degrees: bool     default: False
        If 'True', then angle parameter is assumed to be in degrees
        else it is by default assumed to be in radians
    
    Returns:
    - rot_mat: sp.Matrix        shape: (3, 3)
        The 3x3 rotation matrix for Pitch by angle_rad
    """
    angle_rad = angle if not degrees else sp.rad(angle)
    rot_mat = sp.Matrix([
        [sp.cos(angle_rad), 0, sp.sin(angle_rad)],
        [0, 1, 0],
        [-sp.sin(angle_rad), 0, sp.cos(angle_rad)],
    ])
    return rot_mat

# Rotate about Z Axis
def RotZ(angle, degrees = False):
    """
    Generates a Rotation matrice when the rotation is about Z axis
    A general rotation about Z (Yaw) is given by

                | cT   -sT   0 |
    RotZ(T) =   | sT    cT   0 |
                |  0     0   1 |

    Where T is rotation angle in radians

    Parameters:
    - angle: Symbol or float
        The angle of rotation
    - degrees: bool     default: False
        If 'True', then angle parameter is assumed to be in degrees
        else it is by default assumed to be in radians
    
    Returns:
    - rot_mat: sp.Matrix        shape: (3, 3)
        The 3x3 rotation matrix for Yaw by angle_rad
    """
    angle_rad = angle if not degrees else sp.rad(angle)
    rot_mat = sp.Matrix([
        [sp.cos(angle_rad), -sp.sin(angle_rad), 0],
        [sp.sin(angle_rad), sp.cos(angle_rad), 0],
        [0, 0, 1],
    ])
    return rot_mat

Use the above functions to create a symbolic equation. The output for variable `eu_xyz` should be

$$ \left[\begin{matrix}\cos{\left(\beta \right)} \cos{\left(\gamma \right)} & - \sin{\left(\gamma \right)} \cos{\left(\beta \right)} & \sin{\left(\beta \right)}\\\sin{\left(\alpha \right)} \sin{\left(\beta \right)} \cos{\left(\gamma \right)} + \sin{\left(\gamma \right)} \cos{\left(\alpha \right)} & - \sin{\left(\alpha \right)} \sin{\left(\beta \right)} \sin{\left(\gamma \right)} + \cos{\left(\alpha \right)} \cos{\left(\gamma \right)} & - \sin{\left(\alpha \right)} \cos{\left(\beta \right)}\\\sin{\left(\alpha \right)} \sin{\left(\gamma \right)} - \sin{\left(\beta \right)} \cos{\left(\alpha \right)} \cos{\left(\gamma \right)} & \sin{\left(\alpha \right)} \cos{\left(\gamma \right)} + \sin{\left(\beta \right)} \sin{\left(\gamma \right)} \cos{\left(\alpha \right)} & \cos{\left(\alpha \right)} \cos{\left(\beta \right)}\end{matrix}\right] $$

In [15]:
# Angle symbols
al, be, ga = sp.symbols(r"\alpha, \beta, \gamma")
eu_xyz = RotX(al) * RotY(be) * RotZ(ga)
eu_xyz

Matrix([
[                                      cos(\beta)*cos(\gamma),                                       -sin(\gamma)*cos(\beta),              sin(\beta)],
[sin(\alpha)*sin(\beta)*cos(\gamma) + sin(\gamma)*cos(\alpha), -sin(\alpha)*sin(\beta)*sin(\gamma) + cos(\alpha)*cos(\gamma), -sin(\alpha)*cos(\beta)],
[sin(\alpha)*sin(\gamma) - sin(\beta)*cos(\alpha)*cos(\gamma),  sin(\alpha)*cos(\gamma) + sin(\beta)*sin(\gamma)*cos(\alpha),  cos(\alpha)*cos(\beta)]])

### Answer 2: Solve for angles using **fsolve** from scipy

Let's start by importing scipy

In [16]:
from scipy.optimize import fsolve

Solving the rotation matrix

- Use [scipy.optimize.fsolve](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fsolve.html)
- You could also use [sympy.solvers.solve](https://docs.sympy.org/latest/modules/solvers/solvers.html), set `check=False` (argument) and manually check each solution using [np.allclose](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html#numpy.allclose) (substitute the solution and compare with the real matrix, **M** or **N**)

Using some analysis from the above equation, we can have the following formulas

$$ \alpha = \textup{arctan2} \left( -R_{23},R_{33} \right ) $$
$$ \beta = \textup{arctan2} \left( R_{13}, \sqrt{1-R_{13}^{2}} \right ) $$
$$ \gamma = \textup{arctan2} \left( -R_{12}, R_{11} \right ) $$

Hopefully, we won't run into a case when $\beta = 90 \: \textup{deg}$, because then $\alpha$ and $\beta$ cannot be resolved (the arguments to $\textup{arctan2}$ will become $0$). We can use the above three equations to solve for angles.

In [17]:
# Original Matrices
M_abg = [
    [ 0.26200263, -0.19674724,  0.944799],
    [ 0.21984631,  0.96542533,  0.14007684],
    [-0.93969262,  0.17101007,  0.29619813]
]
M_np = np.array(M_abg, dtype=float)
M_sp = sp.Matrix(M_np)
N_abg = [
    [ 0.        , -0.17364818,  0.98480775],
    [ 0.        ,  0.98480775,  0.17364818],
    [-1.        ,  0.        ,  0.        ]
]
N_np = np.array(N_abg, dtype=float)
N_sp = sp.Matrix(N_np)

In [18]:
# Function of equations
def func_solve_eu_m(eu_angs, mat):
    ax = eu_angs[0]
    ay = eu_angs[1]
    az = eu_angs[2]
    eqs = [
        ax - np.arctan2(-mat[1,2], mat[2,2]),
        ay - np.arctan2(mat[0,2], (1-mat[0,2]**2)**0.5),
        az - np.arctan2(-mat[0,1], mat[0,0])
    ]
    return eqs
# Solve for M
init_vals_m = [ # Put all the initial guess values here (in radians)
    [3, 2, 4],
    [0.5, 0.5, 0.5],
    [-1, -1, 2]
]
print("Roots for M")
for init_val in init_vals_m:
    root = fsolve(func_solve_eu_m, init_val, M_np)
    print(f"\tRoot with init value {init_val}")
    print(f"\t\tRadians: {np.round(root, 3)}")
    print(f"\t\tDegrees: {np.round(np.rad2deg(root), 3)}")
# Solve for N
init_vals_n = [
    [1.5, 2, 5],
    [5, 1.5, 1.5],
    [2, 4, 5]
]
print("Roots for N")
for init_val in init_vals_n:
    root = fsolve(func_solve_eu_m, init_val, N_np)
    print(f"\tRoot with init value {init_val}")
    print(f"\t\tRadians: {np.round(root, 3)}")
    print(f"\t\tDegrees: {np.round(np.rad2deg(root), 3)}")


Roots for M
	Root with init value [3, 2, 4]
		Radians: [-0.442  1.237  0.644]
		Degrees: [-25.31   70.874  36.904]
	Root with init value [0.5, 0.5, 0.5]
		Radians: [-0.442  1.237  0.644]
		Degrees: [-25.31   70.874  36.904]
	Root with init value [-1, -1, 2]
		Radians: [-0.442  1.237  0.644]
		Degrees: [-25.31   70.874  36.904]
Roots for N
	Root with init value [1.5, 2, 5]
		Radians: [-1.571  1.396  1.571]
		Degrees: [-90.  80.  90.]
	Root with init value [5, 1.5, 1.5]
		Radians: [-1.571  1.396  1.571]
		Degrees: [-90.  80.  90.]
	Root with init value [2, 4, 5]
		Radians: [-1.571  1.396  1.571]
		Degrees: [-90.  80.  90.]


In [19]:
# Solution using sympy.solve
# -- For M --
solutions = sp.solvers.solve(sp.Eq(eu_xyz, M_sp), al, be, ga, check=False)
true_solutions = []
for (ax, ay, az) in solutions:
    lhs_mat = np.array(eu_xyz.subs({al: ax, be: ay, ga: az}), dtype=float)
    if np.allclose(lhs_mat, M_np):
        true_solutions.append([ax, ay, az])
sol_m = np.array(copy.deepcopy(true_solutions), dtype=float)   # Solutions for M
print("M can be solved using (XYZ)")
for (i, sol) in enumerate(sol_m):
    print(f"\tSolution {i+1}")
    print(f"\t\t(radians): {np.round(sol, 3)}")
    print(f"\t\t(degrees): {np.round(np.rad2deg(sol), 3)}")
# -- For N --
solutions = sp.solvers.solve(sp.Eq(eu_xyz, N_sp), al, be, ga, check=False)
true_solutions = []
for (ax, ay, az) in solutions:
    lhs_mat = np.array(eu_xyz.subs({al: ax, be: ay, ga: az}), dtype=float)
    if np.allclose(lhs_mat, N_np):
        true_solutions.append([ax, ay, az])
sol_n = np.array(copy.deepcopy(true_solutions), dtype=float)   # Solutions for N
print("N can be solved using (XYZ)")
for (i, sol) in enumerate(sol_n):
    print(f"\tSolution {i+1}")
    print(f"\t\t(radians): {np.round(sol, 3)}")
    print(f"\t\t(degrees): {np.round(np.rad2deg(sol), 3)}")

M can be solved using (XYZ)
	Solution 1
		(radians): [-0.442  1.237  0.644]
		(degrees): [-25.31   70.874  36.904]
	Solution 2
		(radians): [2.7   1.905 3.786]
		(degrees): [154.69  109.126 216.904]
N can be solved using (XYZ)
	Solution 1
		(radians): [1.571 1.745 4.712]
		(degrees): [ 90. 100. 270.]
	Solution 2
		(radians): [4.712 1.396 1.571]
		(degrees): [270.  80.  90.]


The following values will satisfy for **M** (all angles in degrees)

| **Alpha** | **Beta** | **Gamma** |
| :--- | :--- | :---- |
| -25.31 | 70.874  | 36.904  |
| 154.69 | 109.126 | 216.904 |

The following values will satisfy for **N** (all angles in degrees)

| **Alpha** | **Beta** | **Gamma** |
| :--- | :--- | :---- |
| 90.  | 100. | 270. |
| 270. | 80.  | 90.  |


### Answer 3: Gimbal Lock

Mathematical definition and explanation through code



Mathematically, it's when resolving orientations from the rotation matrix is not feasible. For instance, the **Euler XYZ** rotation matrix (as derived earlier) is given by

$$ \left[\begin{matrix}\cos{\left(\beta \right)} \cos{\left(\gamma \right)} & - \sin{\left(\gamma \right)} \cos{\left(\beta \right)} & \sin{\left(\beta \right)}\\\sin{\left(\alpha \right)} \sin{\left(\beta \right)} \cos{\left(\gamma \right)} + \sin{\left(\gamma \right)} \cos{\left(\alpha \right)} & - \sin{\left(\alpha \right)} \sin{\left(\beta \right)} \sin{\left(\gamma \right)} + \cos{\left(\alpha \right)} \cos{\left(\gamma \right)} & - \sin{\left(\alpha \right)} \cos{\left(\beta \right)}\\\sin{\left(\alpha \right)} \sin{\left(\gamma \right)} - \sin{\left(\beta \right)} \cos{\left(\alpha \right)} \cos{\left(\gamma \right)} & \sin{\left(\alpha \right)} \cos{\left(\gamma \right)} + \sin{\left(\beta \right)} \sin{\left(\gamma \right)} \cos{\left(\alpha \right)} & \cos{\left(\alpha \right)} \cos{\left(\beta \right)}\end{matrix}\right] $$

Which is also the same as **Fixed ZYX**. Substituting $\beta = \frac{\pi}{2}$, we get

$$ \left[\begin{matrix}0 & 0 & 1\\\sin{\left(\alpha \right)} \cos{\left(\gamma \right)} + \sin{\left(\gamma \right)} \cos{\left(\alpha \right)} & - \sin{\left(\alpha \right)} \sin{\left(\gamma \right)} + \cos{\left(\alpha \right)} \cos{\left(\gamma \right)} & 0\\\sin{\left(\alpha \right)} \sin{\left(\gamma \right)} - \cos{\left(\alpha \right)} \cos{\left(\gamma \right)} & \sin{\left(\alpha \right)} \cos{\left(\gamma \right)} + \sin{\left(\gamma \right)} \cos{\left(\alpha \right)} & 0\end{matrix}\right] $$

Resolving $\alpha$ and $\gamma$ is now impossible, because the above matrix is essentially

$$ \left[\begin{matrix}0 & 0 & 1\\\sin{\left(\alpha + \gamma \right)} & \cos{\left(\alpha + \gamma \right)} & 0\\- \cos{\left(\alpha + \gamma \right)} & \sin{\left(\alpha + \gamma \right)} & 0\end{matrix}\right] $$

This should be verified by the code block below

In [20]:
# Substitute beta as pi/2 and simplify result
sp.simplify(eu_xyz.subs({be:sp.pi/2}))

Matrix([
[                    0,                    0, 1],
[ sin(\alpha + \gamma), cos(\alpha + \gamma), 0],
[-cos(\alpha + \gamma), sin(\alpha + \gamma), 0]])

We can therefore only find $\alpha +\gamma$ (not the individual angles). This also means that their _individual information_ is lost (singularity!), only the $\alpha + \gamma$ value decides the final orientation. 

Consider the below cases:
1. We first rotate by `10` degrees about X, then cause a singularity by rotating `90` degrees about Y, then rotate by `30` degrees about Z (all axis rotations are in local frame).
2. We first rotate by `0` degrees about X (no movement), then cause a singularity by rotating `90` degrees about Y, then rotate by `40` degrees about Z (all axis rotations are in local frame).
3. We first rotate by `30` degrees about X, then cause a singularity by rotating `90` degrees about Y, then rotate by `10` degrees about Z (all axis rotations are in local frame).
4. We first rotate by `40` degrees about X, then cause a singularity by rotating `90` degrees about Y, then rotate by `0` degrees (no movement) about Z (all axis rotations are in local frame).

**All** the above cases have the **exact same** outcome, which when given to a system, cannot be distinguished. This can be verified by running the cell below


In [21]:
eu_xyz_d = RotX(al, True) * RotY(be, True) * RotZ(ga, True)
r_case1 = np.array(eu_xyz_d.subs({al:10, be:90, ga:30}), dtype=float)
r_case2 = np.array(eu_xyz_d.subs({al:0, be:90, ga:40}), dtype=float)
r_case3 = np.array(eu_xyz_d.subs({al:30, be:90, ga:10}), dtype=float)
r_case4 = np.array(eu_xyz_d.subs({al:40, be:90, ga:0}), dtype=float)
if np.allclose(r_case1, r_case2) and np.allclose(r_case2, r_case3) \
    and np.allclose(r_case3, r_case4):
    print("All cases yield the same rotation matrix!")


All cases yield the same rotation matrix!


A similar problem happens when we substitute $\beta = -\frac{\pi}{2}$ and with **every** three angle representation of rotations. This is the problem with **explicit parameterization** of rotations. To preserve the essence of rotation (to know what actually happened), we use **implicit parameterization**, like quaternions.

### Answer 4: Visualizing Gimbal Lock through Bunny

Let's start by importing the original bunny file

In [22]:
# Read the mesh file
file_name = "./data/bunny.ply"
bunny_mesh = o3d.io.read_triangle_mesh(file_name)
# Convert mesh to point cloud
N = 2500
c = [252, 123, 3]   # Color of point cloud
bunny_pc = bunny_mesh.sample_points_uniformly(N)
bunny_pc.paint_uniform_color([c[0]/255, c[1]/255, c[2]/255])
# A frame (for reference)
ref_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2, origin=[0,0,0])
custom_draw_geometry([bunny_pc, ref_frame])

The **XYZ Euler** angles are the same as **ZYX Fixed** rotation angles, that is, instead of rotating about X, then Y, then Z local axis; we rotate about Z, then Y, and then X global / fixed axis. The result is the same (both have the same rotation matrix for $\alpha$, $\beta$ and $\gamma$ being angles for X,Y and Z respectively).

We compare the following two cases in animation
1. Rotate about global (fixed) Z by `30` degrees, then global Y by `90` degrees and then global X by `10` degrees
2. Rotate about global (fixed) Z by `10` degrees, then global Y by `90` degrees and then global X by `30` degrees

We should observe that the final pose of both the above cases is the same. Therefore showing that individual rotations (about global Z and X) cannot be resolved because of the singularity

In [23]:
# Fixed rotations (Case 1)
bunny_case1 = copy.deepcopy(bunny_pc)
c = [63, 207, 50]   # Color of point cloud (case 1)
bunny_case1.paint_uniform_color([c[0]/255, c[1]/255, c[2]/255])
bunny_case1.rotate(np.array(RotZ(30, True), dtype=float), [0, 0, 0])
bunny_case1.rotate(np.array(RotY(sp.pi/2), dtype=float), [0, 0, 0])
bunny_case1.rotate(np.array(RotX(10, True), dtype=float), [0, 0, 0])
# Fixed rotations (Case 2)
bunny_case2 = copy.deepcopy(bunny_pc)
c = [46, 60, 219]   # Color of point cloud (case 2)
bunny_case2.paint_uniform_color([c[0]/255, c[1]/255, c[2]/255])
bunny_case2.rotate(np.array(RotZ(10, True), dtype=float), [0, 0, 0])
bunny_case2.rotate(np.array(RotY(sp.pi/2), dtype=float), [0, 0, 0])
bunny_case2.rotate(np.array(RotX(30, True), dtype=float), [0, 0, 0])
# Both are the same
if np.allclose(np.array(bunny_case1.points, float), \
    np.array(bunny_case2.points, float)):
    print("Both point clouds are same (result may be superimposed)")

Both point clouds are same (result may be superimposed)


Let's visualize the transformations
- Use the [non-blocking](http://www.open3d.org/docs/release/tutorial/visualization/non_blocking_visualization.html) visualization features of Open3D

In [24]:
import time

In [25]:
def visualize_pc_ZYX_rot(pc: o3d.geometry.PointCloud, az: float, ay: float, \
    ax: float, deg = False, static_gs = None, **kwargs):
    """
    Visualizes the point cloud rotation through ZYX fixed rotations (same result 
    as XYZ Euler angles). Creates an animation window that shows the animation.
    This function captures the execution till the animation is over. A few notes
    about the animation:
    - All rotations happen w.r.t. center as [0, 0, 0]

    Parameters:
    - pc: o3d.geometry.PointCloud
        The point cloud which will be visualized. It is copied (transformations
        are not applied to the point cloud passed). All properties are deeply
        copied.
    - az: float
        Angle of rotation about the Z axis.
    - ay: float
        Angle of rotation about the Y axis.
    - ax: float
        Angle of rotation about the X axis.
    - deg: bool     default: False
        Interpret the angles passed as if they're in degrees (not radians). If 
        'ax', 'ay' and 'az' passed to the function are in radians, then this
        must be 'False', else if they're in degres, then this must be 'True'.
    - static_gs: list of o3d.geometry.Geometry objects  default = None
        A list of Geometry objects that can be displayed in the visualization
        window. You can pass coordinate frames, other point clouds, etc.
    - **kwargs: unwrapped dictionary (keyword arguments)
        - look_at: list (len=3)     default: [0.1, 0.1, 0]
            Set camera look_at for visualization
        - cam_front: list (len=3)   default: [0, 0, 1]
            Set camera front vector
        - cam_up: list (len=3)      default: [0, 1, 0]
            Set camera up vector
        - viz_title: str        default: "Animation"
            The title of animation window
        - sz: float     default: 5.0
            Default step size for Z rotation (in degrees)
        - sy: float     default: 5.0
            Default step size for Y rotation (in degrees)
        - sx: float     default: 5.0
            Default step size for Z rotation (in degrees)
        - save_imgs: bool   default: False
            If True, it saves images in 'save_path' variable. Directories
            have to be existing beforehand (they're not created)
        - save_path: bool   default: "./results/"
            The path / folder where the images are saved

    Returns:
    - final_pc: o3d.geometry.PointCloud
        The final point cloud after all rotations are applied to it
    """
    # Animation objects
    pctf: o3d.geometry.PointCloud = copy.deepcopy(pc)   # Point Cloud to be transformed
    # Some properties of animation
    sz = 5.0 if "sz" not in kwargs else kwargs["sz"]
    step_size_z = float(np.deg2rad(sz))   # Step size (increment angle) in radians
    del_time_z = 0.25  # Delay time in seconds
    sy = 5.0 if "sy" not in kwargs else kwargs["sy"]
    step_size_y = float(np.deg2rad(sy))  # Step size for Rot Y animation
    del_time_y = 0.25   # Delay time for Y animation
    sx = 5.0 if "sx" not in kwargs else kwargs["sx"]
    step_size_x = float(np.deg2rad(sx))  # Step size for Rot X animation
    del_time_x = 0.25   # Delay time for X animation
    end_wait_time = 5  # Wait time in the end (before destroying window) in sec
    save_path = "./results/" if "save_path" not in kwargs else kwargs["save_path"]
    save_imgs = False if "save_imgs" not in kwargs else kwargs["save_imgs"]
    # Some properties of visualization window
    viz_title = "Animation" if "viz_title" not in kwargs else kwargs["viz_title"]
    viz_width = 1080
    viz_height = 720
    look_at = [0.1, 0.1, 0] if "look_at" not in kwargs else kwargs["look_at"]
    cam_front = [0, 0, 1] if "cam_front" not in kwargs else kwargs["cam_front"]
    cam_up = [0, 1, 0] if "cam_up" not in kwargs else kwargs["cam_up"]
    # Some variables to not calculate every loop
    rot_z = np.array(RotZ(step_size_z), dtype=float)
    rot_y = np.array(RotY(step_size_y), dtype=float)
    rot_x = np.array(RotX(step_size_x), dtype=float)
    # -- Start non-blocking visualization --
    vis = o3d.visualization.Visualizer()
    vis.create_window(viz_title, viz_width, viz_height)
    vis.add_geometry(pctf)
    # Add all static geometries
    if static_gs is not None:
        for static_geo in static_gs:
            vis.add_geometry(static_geo)
    # Set view
    ctr = vis.get_view_control()
    ctr.set_lookat(look_at)
    ctr.set_front(cam_front)
    ctr.set_up(cam_up)
    vis.poll_events()
    vis.update_renderer()
    if save_imgs:
        vis.capture_screen_image(f"{save_path}/{viz_title}_start.jpg")
    # Rotate about Z axis
    ar_z = float(np.deg2rad(az)) if deg else float(az)
    steps_z = int(np.around(ar_z/step_size_z))
    for i in range(steps_z):
        # Rotate by step
        pctf.rotate(rot_z, center=[0, 0, 0])
        # Render
        vis.update_geometry(pctf)
        st_time = time.time()
        while (time.time() - st_time < del_time_z):
            vis.poll_events()
            vis.update_renderer()
        if save_imgs:
            vis.capture_screen_image(f"{save_path}/{viz_title}_rz_{i}.jpg")
    # Rotate about Y axis
    ar_y = float(np.deg2rad(ay)) if deg else float(ay)
    steps_y = int(np.around(ar_y/step_size_y))
    for i in range(steps_y):
        # Rotate by step
        pctf.rotate(rot_y, center=[0, 0, 0])
        # Render
        vis.update_geometry(pctf)
        st_time = time.time()
        while (time.time() - st_time < del_time_y):
            vis.poll_events()
            vis.update_renderer()
        if save_imgs:
            vis.capture_screen_image(f"{save_path}/{viz_title}_ry_{i}.jpg")
    # Rotate about X axis
    ar_x = float(np.deg2rad(ax)) if deg else float(ax)
    steps_x = int(np.around(ar_x/step_size_x))
    for i in range(steps_x):
        # Rotate by step
        pctf.rotate(rot_x, center=[0, 0, 0])
        # Render
        vis.update_geometry(pctf)
        st_time = time.time()
        while (time.time() - st_time < del_time_x):
            vis.poll_events()
            vis.update_renderer()
        if save_imgs:
            vis.capture_screen_image(f"{save_path}/{viz_title}_rx_{i}.jpg")
    # Destroy everything
    st_time = time.time()
    while (time.time() - st_time < end_wait_time):
        vis.poll_events()
        vis.update_renderer()
    if save_imgs:
        vis.capture_screen_image(f"{save_path}/{viz_title}_end.jpg")
    vis.destroy_window()
    # Return things
    return pctf

In [26]:
# Animations for case 1 (ZYX = 30, 90, 10)
bunny_case1 = copy.deepcopy(bunny_pc)
c = [63, 207, 50]   # Color of point cloud (case 1)
bunny_case1.paint_uniform_color([c[0]/255, c[1]/255, c[2]/255])
bunny_case1 = visualize_pc_ZYX_rot(bunny_case1, 30, 90, 10, True, \
    static_gs=[ref_frame], \
    look_at=[0, 0.05, 0.05], cam_front=[0.9, 0.3, 0.3], \
    viz_title="Case1", save_imgs=False, save_path="./results/2/imgs")

# Animations for case 2 (ZYX = 10, 90, 30)
bunny_case2 = copy.deepcopy(bunny_pc)
c = [46, 60, 219]   # Color of point cloud (case 2)
bunny_case2.paint_uniform_color([c[0]/255, c[1]/255, c[2]/255])
bunny_case2 = visualize_pc_ZYX_rot(bunny_case2, 10, 90, 30, True, \
    static_gs=[ref_frame], \
    look_at=[0, 0.05, 0.05], cam_front=[0.9, 0.3, 0.3], \
    viz_title="Case2", save_imgs=False, save_path="./results/2/imgs")

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


Comparing different outputs

To save the images, set `save_imgs=True` in the function calls above.

The **Case 1** output looks like

![A](./results/2/Case1.gif)

The **Case 2** output looks like

![B](./results/2/Case2.gif)

GIF made using [this app on Windows 10](https://www.microsoft.com/en-us/p/gif-maker-gif-editor/9pkc9pxzxg9r?activetab=pivot:overviewtab)

In [27]:
# Both are the same finally
if np.allclose(np.array(bunny_case1.points, float), \
    np.array(bunny_case2.points, float)):
    print("Both point clouds are same")

Both point clouds are same


## b) Quaternions

1. What makes Quaternions popular in graphics? 
2. Convert a rotation matrix to quaternion and vice versa. Do not use inbuilt libraries for this question.
3. Perform matrix multiplication of two $\mathcal{R}_{3 \times 3}$ rotation matrices and perform the same transformation in the quaternion space. Verify if the final transformation obtained in both the cases are the same.
4. Try to interpolate any 3D model (cube / bunny / not sphere obviously!!) between two rotation matrices and visualize!

The above questions require you to **code your own functions** and **only verify** using inbuilt functions.

### Answer 1: Quaternions in Graphics

#### Intuition

The concept of quaternions comes from complex numbers. Complex number multiplication can be thought of as rotation of a vector in a 2D plane. Consider the complex number multiplication below

$$ ( a + \mathbf{i} b ) ( c + \mathbf{i} d ) = (ac-bd) + \mathbf{i} (bc+ad) $$

This can be visualized as a matrix multiplication

$$ ( a + \mathbf{i} b ) ( c + \mathbf{i} d ) = \begin{bmatrix}
a & -b \\
b & a
\end{bmatrix} \begin{bmatrix} 
c \\
d
\end{bmatrix} $$

If $(a+\mathbf{i}b) = \cos (\theta) + \mathbf{i} \sin (\theta)$ the above matrix becomes a rotation matrix (note that the complex number still has two numbers, but one constraint)

$$ \mathbf{R} = \begin{bmatrix}
\cos (\theta) & -\sin (\theta) \\
\sin (\theta) & \cos (\theta)
\end{bmatrix} \begin{bmatrix}
c \\
d
\end{bmatrix} $$

Quaternions extend this logic to rotations in three dimensions. As observed earlier, euler angles (which are explicit parameters) run into singularity issues. This makes interpolation and resolving difficult. Quaternions allow representation of rotations using four numbers, with one constraint. Say there is a rotation about the unit vector $\vec{u} = u_x \mathbf{i} + u_y \mathbf{j} + u_z \mathbf{k}$ by angle of $\theta$, the quaternion representation is given by

$$ \mathbf{q} = e^{\frac{\theta}{2} \left( u_x \mathbf{i} + u_y \mathbf{j} + u_z \mathbf{k} \right) } = \cos \frac{\theta}{2} + \left( u_x \mathbf{i} + u_y \mathbf{j} + u_z \mathbf{k} \right) \sin \frac{\theta}{2} $$

The multiplication rules are $\mathbf{i}^2 = \mathbf{j}^2 = \mathbf{k}^2 = \mathbf{i} \mathbf{j} \mathbf{k} = -1$ and $\mathbf{i} \mathbf{j} = \mathbf{k}$, $\mathbf{j} \mathbf{k} = \mathbf{i}$, $\mathbf{k} \mathbf{i} = \mathbf{j}$. This makes programming their multiplication simpler (than even matrix multiplication). This, along with the fact that they don't have singularity problems, makes them a good tool to express orientations (and therefore a popular tool in graphics). However, most of the times, they're internally abstracted.

#### Reference

- [eater.net/quaternions](https://eater.net/quaternions)
- 3Blue1Brown YouTube videos
    - [Visualizing quaternions as a projection](https://www.youtube.com/watch?v=d4EgbgTm0Bg)
    - [Rotations using quaternions](https://www.youtube.com/watch?v=zjMuIxRvygQ&t=4s)

### Answer 2: Matrix and Quaternion conversions

**Convert quaternion to rotation matrix**

Using Rodrigues' rotations formula, the rotation matrix when a rotation about unit vector $\mathbf{k}$ by angle $\theta$ (using right hand thumb rule) is given by

$$ \mathbf{R} = \mathbf{I} + (\sin \theta) \mathbf{[k]} + (1-\cos \theta) \mathbf{[k]}^2 $$

Where 
$$ 
\mathbf{[k]} = \begin{bmatrix}
0 & -k_z & k_y \\
k_z & 0 & -k_x \\
-k_y & k_x & 0 
\end{bmatrix} 
\;\; \textup{and} \;\;
\mathbf{[k]}^2 = \begin{bmatrix}
-k_y^2-k_z^2 & k_x k_y & k_x k_z \\
k_y k_x & -k_x^2-k_z^2 & k_y k_z \\
k_z k_x & k_z k_y & -k_x^2-k_y^2
\end{bmatrix}
$$

Using a quaternion notation $ \mathbf{q} = \left[ k_x \sin \left( \frac{\theta}{2} \right ), \; k_y \sin \left( \frac{\theta}{2} \right ), \; k_z \sin \left( \frac{\theta}{2} \right ), \; \cos \left( \frac{\theta}{2} \right ) \right ] = [q_1, q_2, q_3, q_4] $ the above matrix $\mathbf{R}$ can be reduced. We use $(1-\cos \theta) = 2 \sin^2 \frac{\theta}{2}$ and $\sin \theta = 2 \sin \left( \frac{\theta}{2} \right ) \cos \left( \frac{\theta}{2} \right )$. We get the following result

$$
\mathbf{R} = \begin{bmatrix}
1-2q_2^2-2q_3^2 & 2 (q_1 q_2 - q_3 q_4) & 2 (q_1 q_3 + q_2 q_4) \\
2 (q_2 q_1 + q_3 q_4) & 1-2q_1^2-2q_3^2 & 2 (q_2 q_3 - q_1 q_4) \\
2 (q_3 q_1 - q_2 q_4) & 2 (q_2 q_3 + q_1 q_4) & 1-2q_1^2-2q_2^2
\end{bmatrix}
$$

This can be programmed and tested

In [28]:
# Function
def quat_to_rm(qx, qy, qz, qw):
    """
    Convert a quaternion to a 3x3 rotation matrix.The convention
    used to represent quaternions is
        [ux*sin(th/2), uy*sin(th/2), uz*sin(th/2), cos(th/2)]

    Paraemters:
    - qx: float
        Quaternion k_x * sin(th/2)
    - qy: float
        Quaternion k_y * sin(th/2)
    - qz: float
        Quaternion k_z * sin(th/2)
    - qw: float
        Quaternion cos(th/2)
    
    Returns:
    - rot_m: np.ndarray     shape: (3, 3)   dtype: float
        Rotation matrix constructed using the quaternions
    """
    q1 = float(qx)
    q2 = float(qy)
    q3 = float(qz)
    q4 = float(qw)
    rot_m = np.array([
        [1-2*(q2**2)-2*(q3**2), 2*(q1*q2-q3*q4), 2*(q1*q3+q2*q4)],
        [2*(q2*q1+q3*q4), 1-2*(q1**2)-2*(q3**2), 2*(q2*q3-q1*q4)],
        [2*(q3*q1-q2*q4), 2*(q2*q3+q1*q4), (1-2*(q1)**2-2*(q2)**2)]
    ], dtype=float)
    return rot_m

In [29]:
# Testing quaternion to rotation matrix
r_actual = np.array(RotX(sp.pi/3), dtype=float)
r = quat_to_rm(1*np.sin(0.5*np.pi/3), 0, 0, np.cos(0.5*np.pi/3))
if np.allclose(r, r_actual):
    print("Works for RotX")
r_actual = np.array(RotY(sp.pi/2), dtype=float)
r = quat_to_rm(0, 1*np.sin(0.5*np.pi/2), 0, np.cos(0.5*np.pi/2))
if np.allclose(r, r_actual):
    print("Works for RotY")
r_actual = np.array(RotZ(sp.pi/6), dtype=float)
r = quat_to_rm(0, 0, 1*np.sin(0.5*np.pi/6), np.cos(0.5*np.pi/6))
if np.allclose(r, r_actual):
    print("Works for RotZ")

Works for RotX
Works for RotY
Works for RotZ


**Convert rotation matrix to quaternion**

Consider

$$
\mathbf{R} = \begin{bmatrix}
r_{11} & r_{12} & r_{13} \\
r_{21} & r_{22} & r_{23} \\
r_{31} & r_{32} & r_{33}
\end{bmatrix}
$$

We need to get quaternion $ \mathbf{q} = \left[ k_x \sin \left( \frac{\theta}{2} \right ), \; k_y \sin \left( \frac{\theta}{2} \right ), \; k_z \sin \left( \frac{\theta}{2} \right ), \; \cos \left( \frac{\theta}{2} \right ) \right ] = [q_1, q_2, q_3, q_4] $ from the above matrix. To do that, follow the following steps

1. Calculate $\mathbf{v}$ using the formula below

    $$ 
    \mathbf{v} = \begin{bmatrix}
    v_1 \\ v_2 \\ v_3 \\ v_4
    \end{bmatrix} = \begin{bmatrix}
    \frac{1}{2} \sqrt{1+r_{11}-r_{22}-r_{33}} \\
    \frac{1}{2} \sqrt{1-r_{11}+r_{22}-r_{33}} \\
    \frac{1}{2} \sqrt{1-r_{11}-r_{22}+r_{33}} \\
    \frac{1}{2} \sqrt{1+r_{11}+r_{22}+r_{33}}
    \end{bmatrix}
    $$

    For the next step, get the maximum value among $v_1, v_2, v_3, v_4$ (all values of $\mathbf{v}$).

2. If $v_1$ is largest in $\mathbf{v}$, calculate the rest using $q_1 = v_1$. The quaternions are given by

    $$
    \mathbf{q} = \begin{bmatrix}
    q_1 \\ q_2 \\ q_3 \\ q_4
    \end{bmatrix} = \begin{bmatrix}
    v_1 \\
    \frac{r_{12} + r_{21}}{4q_1} \\
    \frac{r_{13} + r_{31}}{4q_1} \\
    \frac{r_{32} - r_{23}}{4q_1}
    \end{bmatrix}
    $$

    This follows that

    $$ q_1 = v_1 = \frac{1}{2} \sqrt{1+r_{11}-r_{22}-r_{33}} $$

3. If $v_2$ is largest in $\mathbf{v}$, calculate the rest using $q_2 = v_2$. The quaternions are given by

    $$
    \mathbf{q} = \begin{bmatrix}
    q_1 \\ q_2 \\ q_3 \\ q_4
    \end{bmatrix} = \begin{bmatrix}
    \frac{r_{12}+r_{21}}{4q_2} \\
    v_2 \\
    \frac{r_{23}+r_{32}}{4q_2} \\
    \frac{r_{13}-r_{31}}{4q_2}
    \end{bmatrix}
    $$

    This follows that

    $$ q_2 = v_2 = \frac{1}{2} \sqrt{1-r_{11}+r_{22}-r_{33}} $$

4. If $v_3$ is largest in $\mathbf{v}$, calculate the rest using $q_3 = v_3$. The quaternions are given by

    $$
    \mathbf{q} = \begin{bmatrix}
    q_1 \\ q_2 \\ q_3 \\ q_4
    \end{bmatrix} = \begin{bmatrix}
    \frac{r_{13}+r_{31}}{4q_3} \\
    \frac{r_{23}+r_{32}}{4q_3} \\
    v_3 \\
    \frac{r_{21}-r_{12}}{4q_3}
    \end{bmatrix}
    $$

    This follows that

    $$ q_3 = v_3 = \frac{1}{2} \sqrt{1-r_{11}-r_{22}+r_{33}} $$

5. If $v_4$ is largest in $\mathbf{v}$, calculate the rest using $q_4 = v_4$. The quaternions are given by

    $$
    \mathbf{q} = \begin{bmatrix}
    q_1 \\ q_2 \\ q_3 \\ q_4
    \end{bmatrix} = \begin{bmatrix}
    \frac{r_{32}-r_{23}}{4q_4} \\
    \frac{r_{13}-r_{31}}{4q_4} \\
    \frac{r_{21}-r_{12}}{4q_4} \\
    v_4 \\
    \end{bmatrix}
    $$

    This follows that

    $$ q_4 = v_4 = \frac{1}{2} \sqrt{1+r_{11}+r_{22}+r_{33}} $$

This has to be programmed

In [30]:
# Function
def rm_to_quat(rot_m):
    """
    Convert a rotation matrix to quaternions. The convention
    used to represent quaternions is
        [ux*sin(th/2), uy*sin(th/2), uz*sin(th/2), cos(th/2)]
    
    Parameters:
    - rot_m: np.ndarray     shape: (3,3)
        A 3x3 rotation matrix
    
    Returns:
    - qx: float
        Quaternion ux*sin(th/2)
    - qy: float
        Quaternion uy*sin(th/2)
    - qz: float
        Quaternion uz*sin(th/2)
    - qw: float
        Quaternion cos(th/2)
    """
    # Parse all elements
    [r11, r12, r13] = rot_m[0, :]
    [r21, r22, r23] = rot_m[1, :]
    [r31, r32, r33] = rot_m[2, :]
    # Get 'v' vector
    v = np.array([
        0.5*((1+r11-r22-r33)**0.5), # v1
        0.5*((1-r11+r22-r33)**0.5), # v2
        0.5*((1-r11-r22+r33)**0.5), # v3
        0.5*((1+r11+r22+r33)**0.5)  # v4
    ])
    mvi = np.argmax(v)  # Maximum value quaternion
    # Returning vector
    qx = 0.0
    qy = 0.0
    qz = 0.0
    qw = 1.0
    if mvi == 0:    # v1
        qx = v[0]
        qy = (r12 + r21)/(4*v[0])
        qz = (r13 + r31)/(4*v[0])
        qw = (r32 - r23)/(4*v[0])
    elif mvi == 1:  # v2
        qx = (r12 + r21)/(4*v[1])
        qy = v[1]
        qz = (r23 + r32)/(4*v[1])
        qw = (r13 - r31)/(4*v[1])
    elif mvi == 2:  # v3
        qx = (r13 + r31)/(4*v[2])
        qy = (r23 + r32)/(4*v[2])
        qz = v[2]
        qw = (r21 - r12)/(4*v[2])
    elif mvi == 3:  # v4
        qx = (r32 - r23)/(4*v[3])
        qy = (r13 - r31)/(4*v[3])
        qz = (r21 - r12)/(4*v[3])
        qw = v[3]
    return [qx, qy, qz, qw]

Use [scipy.spatial.transform.Rotation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.html#scipy.spatial.transform.Rotation) for verification

In [31]:
# Let's verify it this works
from scipy.spatial.transform import Rotation
rot_mat = np.array(RotX(25, True) * RotY(30, True) * RotZ(30, True), dtype=float)
qvals = np.array(rm_to_quat(rot_mat))
qvals_actual = Rotation.from_matrix(rot_mat).as_quat()
if np.allclose(qvals, qvals_actual):
    print("Quaternion logic appears correct")


Quaternion logic appears correct


### Answer 3: Transformations using Quaternions

Perform quaternion multiplication using standard rules of quaternion multiplication. The multiplication rules are $\mathbf{i}^2 = \mathbf{j}^2 = \mathbf{k}^2 = \mathbf{i} \mathbf{j} \mathbf{k} = -1$ and $\mathbf{i} \mathbf{j} = \mathbf{k}$, $\mathbf{j} \mathbf{k} = \mathbf{i}$, $\mathbf{k} \mathbf{i} = \mathbf{j}$. Refer to the image below for graph representation

[![Cayley Q8 quaternion multiplication graph.svg](https://upload.wikimedia.org/wikipedia/commons/0/04/Cayley_Q8_quaternion_multiplication_graph.svg)](https://en.wikipedia.org/wiki/File:Cayley_Q8_quaternion_multiplication_graph.svg)

The result is the [Hamilton product](https://en.wikipedia.org/wiki/Quaternion#Hamilton_product) of quaternions, but using our convention

In [32]:
# Function to multiply two quaternions
def quat_mul(qu1, qu2):
    """
    Perform quaternion multiplication and return the result. The
    multiplication result is qu1 * qu2. The convention used to 
    represent quaternions is
        [ux*sin(th/2), uy*sin(th/2), uz*sin(th/2), cos(th/2)]
    The first three elements are [i,j,k] components and the last
    one is the real component

    Paraemters:
    - qu1: list or np.ndarray   shape: (4,)
        First quaternion
    - qu2: list or np.ndarray   shape: (4,)
        Second quaternion
    
    Returns:
    - qu_res: list      len: 4
        Resultant quaternion which is obtained by qu1 * qu2 after
        applying the rules of quaternion multiplication. Note
        that the result is [qx, qy, qz, qw].
    """
    [q11, q12, q13, q14] = qu1
    [q21, q22, q23, q24] = qu2
    qu_res = [
        q11*q24 + q12*q23 - q13*q22 + q14*q21,  # qx
        -q11*q23 + q12*q24 + q13*q21 + q14*q22, # qy
        q11*q22 - q12*q21 + q13*q24 + q14*q23,  # qz
        q14*q24 - q11*q21 - q12*q22 - q13*q23   # qw
    ]
    return qu_res

In [33]:
# Compare multiplication of rot1 and rot2
rot1 = np.array(RotX(30, True) * RotY(45, True), dtype=float)
qu1 = rm_to_quat(rot1)
rot2 = np.array(RotY(45, True) * RotZ(10, True), dtype=float)
qu2 = rm_to_quat(rot2)
# Rotation matrix multiplication
rot_res = rot1 @ rot2
quatr_res = rm_to_quat(rot_res)
# Quaternion multiplication
qu_res = quat_mul(qu1, qu2)
rotq_res = quat_to_rm(*qu_res)
# Check if close
if np.allclose(rot_res, rotq_res):
    print("Quaternion and rotation matrix multiplication give the same rotation matrices")
if np.allclose(quatr_res, qu_res):
    print("Quaternion and rotation matrix multiplication give the same quaternions")

Quaternion and rotation matrix multiplication give the same rotation matrices
Quaternion and rotation matrix multiplication give the same quaternions


**Inverting quaternions**

This can be done using the [conjugate of the quaternion](https://en.wikipedia.org/wiki/Quaternion#Unit_quaternion).

In [34]:
# Quaternion inverse
def quat_inv(qu):
    """
    Constructs an inverse of quaternion passed. The convention used
    to represent quaternions is
        [ux*sin(th/2), uy*sin(th/2), uz*sin(th/2), cos(th/2)]
    Quaternion has to be of unit length, as the function actually
    returns the conjugate
    
    Paraemters:
    - qu: list or np.ndarray   shape: (4,)
        Quaternion that has to be inverted
    
    Returns:
    - qu_inv: list or np.ndarray   shape: (4,)
        Inverted quaternion. Multiplication of this qith 'qu' should
        give the unit quaternion [0, 0, 0, 1]
    """
    [q1, q2, q3, q4] = qu
    qu_inv = [-q1, -q2, -q3, q4]    # Inverse quaternion
    return qu_inv

In [35]:
# Check if inversion works fine
qu1_inv = quat_inv(qu1)
qu_identity = quat_mul(qu1, qu1_inv)
qu_identity2 = quat_mul(qu1_inv, qu1)
if np.allclose(qu_identity, qu_identity2):
    print("Quaternion inversion works fine")

Quaternion inversion works fine


### Answer 4: Interpolation using Quaternions

Quaternion interpolation requires conservation of the length of quaternions (they must all be unit lengths). The **SLERP** algorithm performs _Spherical Linear Interpolation_ and has the following formula

$$ Slerp \left( t; \mathbf{q}_{i}, \mathbf{q}_{f} \right ) = \frac{\textup{Sin} \left( (1-t) \theta \right )}{\textup{Sin} (\theta)} \mathbf{q}_{i} \: + \: \frac{\textup{Sin} \left( t \theta \right )}{\textup{Sin} (\theta)} \mathbf{q}_{f} $$

Where $ \theta = \textup{cos}^{-1} \left( \mathbf{q}_i \: \cdot \: \mathbf{q}_f \right ) $

**Reference**

- [Geometric Slerp on Wikipedia](https://en.wikipedia.org/wiki/Slerp#Geometric_Slerp)

In [36]:
# Return intermediate quaternions
def gen_intermediate_quats(quat_init, quat_final, N=10):
    """
    Generate intermediate quaternions from an initial to final
    quaternion rotation pose. The convention used to represent 
    quaternions is
        [ux*sin(th/2), uy*sin(th/2), uz*sin(th/2), cos(th/2)]

    The function uses SLERP (Spherical Linear Interpolation)
    to interpolate intermediate points
    
    Parameters:
    - quat_init: list or np.ndarray     shape: (4,)
        Initial quaternion. All values must be normalized and
        floating point (they're case to float)
    - quat_final: list or np.ndarray     shape: (4,)
        Final quaternion. All values must be normalized and
        floating point (they're case to float)
    - N: int    default: 10
        The number of intermediate steps. This is NOT time
        sampled. Assume a time range from 0 to 1, broken into
        N stages
    
    Returns:
    - qu_n_vals: np.ndarray     shape: (N, 4)
        The sequence of quaternions
    """
    # Initial and final quaternions
    qu_i = np.array(quat_init, dtype=float)
    qu_f = np.array(quat_final, dtype=float)
    # Theta values
    theta = np.arccos(np.dot(qu_i, qu_f))
    # Interpolation timestamps
    t = np.linspace(0, 1, N)
    coeffs_init = np.sin((1-t)*theta)/np.sin(theta)
    coeffs_final = np.sin(t*theta)/np.sin(theta)
    # All values
    qu_n_vals = coeffs_init.reshape(-1, 1) * qu_i + coeffs_final.reshape(-1, 1) * qu_f
    return qu_n_vals

In [37]:
# Animation of point cloud between orientations
def visualize_pc_rots(pc: o3d.geometry.PointCloud, qu_i, qu_f, N = 10, static_gs = None, \
    color_init = [46,60,219], color_final = [63,207,50], color_anim = [240,165,53], \
    ):
    """
    Visualizes a point cloud rotating between two given orientations
    as quaternions. The point cloud convention used is as follows
        [ux*sin(th/2), uy*sin(th/2), uz*sin(th/2), cos(th/2)]
    The point cloud of initial and final stages is also displayed


    Parameters:
    - pc: o3d.geometry.PointCloud
        The point cloud which will be visualized. This is not altered
        and a deepcopy is always used.
    - qu_i: list or np.ndarray      shape: (4,)     dtype: float
        Initial orientation as a quaternion (normalized)
    - qu_f: list or np.ndarray      shape: (4,)     dtype: float
        Final orientation as a quaternion (normalized)
    - N: int
        Number of intermediate orientations to show
    - static_gs: list of o3d.geometry.Geometry objects  default = None
        A list of Geometry objects that can be displayed in the 
        visualization window. You can pass coordinate frames, other 
        point clouds, etc.
    - color_init: list      len: 3  default: [46,60,219]
        The color of initial point cloud
    - color_final: list     len: 3  default: [63,207,50]
        The color of final point cloud
    - color_anim: list      len: 3  default: [240,165,53]
        The color of point cloud when in animation (intermediate stages)
    
    """
    # Animation objects and variables
    pc_main = copy.deepcopy(pc)
    pc_init = copy.deepcopy(pc_main)
    pc_final = copy.deepcopy(pc_main)
    pc_inter = copy.deepcopy(pc_main)
    end_wait_time = 1
    step_time = 0.25
    look_at = [0, 0.05, 0.05]
    cam_front = [-1, 0.2, 0.2]
    cam_up = [0, 1, 0]
    # Some variables to be used during animation
    rot_init = quat_to_rm(qu_i[0], qu_i[1], qu_i[2], qu_i[3])
    pc_init.rotate(rot_init, [0, 0, 0])
    rot_final = quat_to_rm(qu_f[0], qu_f[1], qu_f[2], qu_f[3])
    pc_final.rotate(rot_final, [0, 0, 0])
    quat_vals = gen_intermediate_quats(qu_i, qu_f, N)
    # Some properties of visualization window
    viz_title = "Animation"
    viz_width = 1080
    viz_height = 720
    # Some alterations to animation objects
    c = color_init
    pc_init.paint_uniform_color([c[0]/255, c[1]/255, c[2]/255])
    c = color_final
    pc_final.paint_uniform_color([c[0]/255, c[1]/255, c[2]/255])
    c = color_anim
    pc_inter.paint_uniform_color([c[0]/255, c[1]/255, c[2]/255])
    # -- Start non-blocking visualization --
    vis = o3d.visualization.Visualizer()
    vis.create_window(viz_title, viz_width, viz_height)
    if static_gs is not None:
        for static_geo in static_gs:
            vis.add_geometry(static_geo)
    vis.add_geometry(pc_init)
    vis.add_geometry(pc_final)
    vis.add_geometry(pc_inter)  # Intermediate point cloud
    # Set view
    ctr = vis.get_view_control()
    ctr.set_lookat(look_at)
    ctr.set_front(cam_front)
    ctr.set_up(cam_up)
    vis.poll_events()
    vis.update_renderer()
    # Animate through every step
    for i in range(N):
        curr_quat = quat_vals[i, :]
        rot_m = quat_to_rm(curr_quat[0], curr_quat[1], curr_quat[2], curr_quat[3])
        pc_main.rotate(rot_m, [0, 0, 0])
        # Transfer points
        pc_inter.points = pc_main.points
        # Update visualization
        vis.update_geometry(pc_inter)
        pc_main = copy.deepcopy(pc) # Make the copy again
        st_time = time.time()
        while (time.time() - st_time < step_time):
            vis.poll_events()
            vis.update_renderer()
    # Destroy everything
    st_time = time.time()
    while (time.time() - st_time < end_wait_time):
        vis.poll_events()
        vis.update_renderer()
    vis.destroy_window()

In [38]:
# Rotation that has happened
rot = np.array(RotX(30, True) * RotY(90, True) * RotZ(10, True), dtype=float)
qu_i = np.array([0, 0, 0, 1])
qu_f = np.array(rm_to_quat(rot))
# Create a copy of point cloud
bunny_points = copy.deepcopy(bunny_pc)
ref_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2, origin=[0,0,0])
# Show the visualization
visualize_pc_rots(bunny_points, qu_i, qu_f, static_gs=[ref_frame], N=20)

The animation below was achieved with the following python code (for initial and final quaternion values)

```py
rot = np.array(RotX(30, True) * RotY(90, True) * RotZ(10, True), dtype=float)
qu_i = np.array([0, 0, 0, 1])
qu_f = np.array(rm_to_quat(rot))
```

And the individual images were captured using [capture_screen_image](http://www.open3d.org/docs/release/python_api/open3d.visualization.Visualizer.html#open3d.visualization.Visualizer.capture_screen_image) function

![Animation of quaternion interpolation](./results/2/Quat_animation.gif)

GIF made using [this app on Windows 10](https://www.microsoft.com/en-us/p/gif-maker-gif-editor/9pkc9pxzxg9r?activetab=pivot:overviewtab)

## c) Exponential maps (Bonus)

1. What is the idea behind exponential map representation of rotation matrices?
2. Perform matrix exponentiation and obtain the rotation matrix to rotate a vector $P$ around $\omega$ for $\theta$ seconds.
$$
\omega = \begin{bmatrix}2 \\ 1 \\ 15 \end{bmatrix}
$$

$$
\theta = 4.1364
$$

3. Compute the logarithmic map (SO(3) to so(3)) of the rotation matrix to obtain the rotation vector and the angle of rotation
$$
\begin{bmatrix}
0.1 &  -0.9487 & 0.3 \\
0.9487 & 0.  & -0.3162 \\
0.3   &  0.3162  & 0.9 
\end{bmatrix}
$$
You can use inbuilt libraries **only to verify** your results.

### Answer 1: Why Axis-Angle


Exponential maps or **Angle-Axis convention** provide an intuitive understanding of rotations that we can visualize, and that still adhere to the norms of rotational matrices (representing successive operations as multiplication). Since they're an implicit representation, they're not prone to singularities (like how euler angles are, as they're an explicit representation).

The idea is to represent every rotation operation as a single rotation by an angle $\theta$ about a unit vector $\omega$ (using the right hand convention). This is easier to visualize, as vectors can be represented in 3D space and a rotation by $\theta$ can also be visualized intuitively (curl your right hand along the vector, with thumb pointing towards the head, the rotational sense of your right hand fingers show the rotation direction). This gives them a representational advantage over other intrinsic representations like quaternions (that are harder to visualize)

References:
- [Wikipedia](https://en.wikipedia.org/wiki/Axis%E2%80%93angle_representation#Exponential_map_from_%7F'%22%60UNIQ--postMath-00000006-QINU%60%22'%7F(3)_to_SO(3))

### Answer 2: Rotation matrix from Axis-angle

Assuming that angle of rotation is $\theta$, given in radians

Using Rodrigues' rotations formula, the rotation matrix when a rotation about unit vector $\mathbf{k}$ by angle $\theta$ (using right hand thumb rule) is given by

$$ \mathbf{R} = \mathbf{I} + (\sin \theta) \mathbf{[k]} + (1-\cos \theta) \mathbf{[k]}^2 $$

Where 
$$ 
\mathbf{[k]} = \begin{bmatrix}
0 & -k_z & k_y \\
k_z & 0 & -k_x \\
-k_y & k_x & 0 
\end{bmatrix} 
\;\; \textup{and} \;\;
\mathbf{[k]}^2 = \begin{bmatrix}
-k_y^2-k_z^2 & k_x k_y & k_x k_z \\
k_y k_x & -k_x^2-k_z^2 & k_y k_z \\
k_z k_x & k_z k_y & -k_x^2-k_y^2
\end{bmatrix}
$$

We get

$$
\mathbf{R} = \begin{bmatrix}
1+(1-\cos\theta)(k_x^2-1) & -k_z\sin\theta+(1-\cos\theta)k_xk_y & k_y\sin\theta+(1-\cos\theta)k_xk_z \\
k_z\sin\theta+(1-\cos\theta)k_yk_x & 1+(1-\cos\theta)(k_y^2-1) & -k_x\sin\theta+(1-\cos\theta)k_yk_z \\
-k_y\sin\theta+(1-\cos\theta)k_zk_x & k_x\sin\theta+(1-\cos\theta)k_zk_y & 1+(1-\cos\theta)(k_z^2-1) \\
\end{bmatrix}
$$

You can accomplish this using scipy
- Use [Rotation.from_rotvec](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.from_rotvec.html) to construct Rotation object using rotation vector. Note that this takes only three numbers, as the axis is expected to be normalized (so multiply the angle and axis).
- Use [Rotation.as_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.as_matrix.html) to get the rotation matrix back

In [39]:
# Angle axis to rotation matrix
def axang_to_rm(ax, ang, degrees = False):
    """
    Convert axis-angle representation to rotation matrix

    Parameters:
    - ax: list or np.ndarray    shape: (3,)
        The axis of rotation where ax[0] is X coordinate, ax[1] is Y
        coordinate and ax[2] is Z coordinate
    - ang: float
        The angle of rotation
    - degrees: bool     default: False
        If 'True', the 'ang' angle is assumed to be in degrees and is
        subsequently converted to radians. Else if 'False', the 'ang'
        value is assumed to already be in radians
    
    Returns:
    - rot_m: np.ndarray     shape: (3, 3)
        A 3x3 rotation matrix (direction cosines)
    """
    th_rad = np.deg2rad(ang) if degrees else ang
    [kx, ky, kz] = ax[0], ax[1], ax[2]
    k_cross = sp.Matrix([
        [0, -kz, ky],
        [kz, 0, -kx],
        [-ky, kx, 0]
    ])
    k_cross_sq = k_cross * k_cross
    rot_m = np.eye(3) + sp.sin(th_rad) * k_cross + (1-sp.cos(th_rad)) * k_cross_sq
    return rot_m

In [40]:
w_axis = np.array([2, 1, 15])
w_axis = w_axis / np.linalg.norm(w_axis)
ang_th = 4.1364
px, py, pz = sp.symbols('P_x, P_y, P_z')
P_vect = sp.Matrix([[px], [py], [pz]])
rot_m = axang_to_rm(w_axis, ang_th)
# Use scipy to verify
rm = Rotation.from_rotvec(w_axis * ang_th)
# Rotation matrix is
if np.allclose(rm.as_matrix(), np.array(rot_m, dtype=float)):
    print("Rotation matrix is identical through Scipy")
    print(np.array(rot_m, dtype=float))
# Transformation of P
print("The vector P = [Px, Py, Pz] will become")
rot_m * P_vect

Rotation matrix is identical through Scipy
[[-0.51780074  0.84292002  0.14617876]
 [-0.81605629 -0.53794854  0.21133741]
 [ 0.25677718 -0.00985943  0.96642034]]
The vector P = [Px, Py, Pz] will become


Matrix([
[ -0.517800739415721*P_x + 0.842920021873521*P_y + 0.146178763797195*P_z],
[ -0.816056291972358*P_x - 0.537948536841593*P_y + 0.211337408052421*P_z],
[0.256777184720253*P_x - 0.00985943379369655*P_y + 0.966420337623546*P_z]])

### Answer 3: Axis-angle from Rotation matrix

The angle and axis can be derived from the rotation matrix by doing the following

$$
\mathbf{R} = \begin{bmatrix}
r_{11} & r_{12} & r_{13} \\
r_{21} & r_{22} & r_{23} \\
r_{31} & r_{32} & r_{33}
\end{bmatrix}
$$

The angle is given by

$$ \theta = \arccos \left( \frac{r_{11}+r_{22}+r_{33}-1}{2} \right ) $$

The axis is then retrieved using

$$
\mathbf{k} = \frac{1}{2 \sin\theta} \begin{bmatrix}
r_{32} - r_{23} \\
r_{13} - r_{31} \\
r_{21} - r_{12}
\end{bmatrix}
$$

Use the following scipy functions for verification
- Use [Rotation.from_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.from_matrix.html) to construct a Rotation object from rotation matrix
- Use [Rotation.as_rotvec](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.transform.Rotation.as_rotvec.html) to get the rotation vector and angle. The angle is the norm (magnitude) and the axis has to be normalized.

In [41]:
# Rotation matrix to angle axis
def rm_to_axang(rot_m, degrees = False):
    """
    Convert rotation matrix to angle-axis representation

    Parameters:
    - rot_m: np.ndarray     shape: (3, 3)
        Rotation Matrix
    
    Returns:
    - ax: np.ndarray    shape: (3,1)
        The axis
    - ang: float
        The angle. If 'degrees' is False, then in radians. Else
        if 'degrees' is True, then in degrees.
    """
    # Parse elements
    r11 = rot_m[0][0]
    r12 = rot_m[0][1]
    r13 = rot_m[0][2]
    r21 = rot_m[1][0]
    r22 = rot_m[1][1]
    r23 = rot_m[1][2]
    r31 = rot_m[2][0]
    r32 = rot_m[2][1]
    r33 = rot_m[2][2]
    # Angle
    ang_rad = np.arccos((r11+r22+r33-1)/2)
    ang = np.rad2deg(ang_rad) if degrees else ang_rad
    # Axis
    ax = (1/(2*np.sin(ang_rad))) * np.array([
        [r32-r23],
        [r13-r31],
        [r21-r12]
    ])
    return ax, ang

In [42]:
# Rotation matrix
rot_m = np.array([
    [0.1, -0.9487, 0.3],
    [0.9487, 0.0, -0.3162],
    [0.3, 0.3162, 0.9]
])
# Get axis and angle
ax, ang = rm_to_axang(rot_m)
print(f"Axis is \n{ax}")
print(f"Angle is {ang:.4f} (which is {np.rad2deg(ang):.3f} degrees)")
# Testing with scipy
axang_sp = Rotation.from_matrix(rot_m).as_rotvec()
ang_sp = np.linalg.norm(axang_sp)
ax_sp = axang_sp / ang_sp
if np.allclose(ax_sp, ax.flatten()) and np.allclose(ang_sp, ang):
    print("Scipy returns the same axis and angle")

Axis is 
[[0.3162]
 [0.    ]
 [0.9487]]
Angle is 1.5708 (which is 90.000 degrees)
Scipy returns the same axis and angle


# 3. Data representations

## a) Octomaps

1. Why is an Octomap memory efficient?
2. When do we update an Octomap and why?
3. When would you likely use an octomap instead of a point cloud?
 

### Answer 1: Octomap memory

An octomap stores environmental data (usually obtained through a point cloud) in a tree like data structure. Each node is a cube called a voxel which can be further divided into 8 sub-cubes (sub-voxels). Some of the reasons why octomaps are memory efficient are as follows
- The information about the coordinates of each node (center location of each cube) need not be stored (it can be inferred on traversal), only the probabilistic estimate of its occupancy is stored. 
- The depth of the tree is bounded by the maximum resolution of the data. Additionally, segmenting the tree to a particular depth level (less than the leaf nodes) yields a map with coarser resolution. So this allows storing artifact information and retrievals of multiple resolutions.
- If all children of a node have the same state (occupied or free) they can be pruned, to save memory. Such pruning is also possible when constructing or modifying the tree. 
- Using maximum likelihood probabilities before tree pruning leads to even greater compression and lesser memory requirements.

Additionally, you can store extra information for every voxel (like temperature or color of the region)

### Answer 2: Updating Octomaps

Note that occupancy values are updated as **log-odds** values as shown below (it's essentially accumulating previous log-odds estimates and thresholding them)

$$ \textup{L} (n|z_{1:t}) = \textup{max} \left( \textup{min} \left( \textup{L} (n|z_{1:t-1}) + \textup{L}(n|z_{t}), l_{\textup{max}} \right ), l_{\textup{min}} \right ) $$

Where $\textup{L}(n) = \textup{log} \left[ \frac{P(n)}{1-P(n)} \right ]$ (log, odds). In a static environment, all voxels will converge to a stable state ($l_{\textup{min}}$ for free and $l_{\textup{max}}$ for occupied). However, when readings come that change a node / voxel from stable to _unstable_ (new _measurements that contradict_ the state of corresponding node), then its children (which were pruned earlier when the voxel became stable) will have to be **regenerated** and **updated** (resampled and added to the tree) accordingly. The bounds in the above equation are so that the map can adapt to changes in the environment quickly, and to promote compression among stable nodes. Say we didn't have them, we would need as many observations contradicting the current voxel state as many were taken to affirm that state earlier, this would slow the updating process of the map significantly.

Basically, we update the map when contradicting readings are achieved and, because we do not want to loose the sub-voxel data, we update children nodes in the octree as well. We do this to maintain the map's correctness.

### Answer 3: Octomaps Vs. Point Clouds

While point clouds are simpler to visualize, they have serious memory constraints (over a large area that needs to be mapped) and cannot easily be stored in a hierarchical manner. Precisely, you could use octomaps instead of point clouds in the following cases

- Storage and transmission of spatial data in terms of occupancy. Point clouds would take significantly more information (sampled points of 3D space) compared to octrees (using which the octomap is made)
- Hierarchical models where some objects (maybe foreground objects of prominence) could have higher resolution separate octrees compared to less important objects (maybe background objects). Hierarchical octrees allow subtrees to be created, which are very useful in environment recognition. Doing the same through point clouds is difficult.


## b) Signed Distance Functions

1. How do we determine object surfaces using SDF?
2. How do we aggregate views from multiple cameras? (just a general overview is fine)
3. Which preserves details better? Voxels or SDF? Why?
4. What’s an advantage of SDF over a point cloud?


### Answer 1: Object Surfaces using SDF

A Signed Distance Function is applied to individually sampled points in space. Their value is associated to the distance to the nearest surface and the position relative to the surface. Usually, if a point is in the surface (surface comes first in sensor measurement / ray from sensor), then, it is given a positive value. If a point is outside the surface (point comes first in ray from sensor), then, it is given a negative value. So it is high inside surface / object, decreases towards 0 as we move to surface, then further decreases (goes negative) as we move outwards the object surface.

Therefore, an object surface can be extracted from the sign distance function by **sampling the points whose value is 0** (points on the surface).


### Answer 2: Aggregating multiple camera views

A camera essentially projects a 3D scene to a surface. During this process, the depth of the scene is lost. By capturing the scene from multiple views, we can extract several depth features of the scene back from the multiple images. A scene can be reconstructed by aggregating multiple pictures from the same camera, taken from different poses or multiple cameras can be used to do this process live / on-the-fly.

Say we have two cameras that have captured images (say one image from each) of the same scene and we have to find the depth information of the scene. We would roughly follow the following steps

1. Find the **camera parameters**: This stereo setup has several parameters that are divided into two main categories: _intrinsic_ and _extrinsic_ parameters. 

    Let's briefly describe these

    - Intrinsic parameters are parameters of the camera, things like focal length. These parameters allow efficient projection lines in 3D space. That is, given a pixel in the image, project it's ray in 3D space (a set of points where that pixel could be in 3D space) that could have made that pixel land where it is in the image. These parameters are particular to the camera. This basically gives the mapping from 3D world coordinates to 2D image coordinates.
    - Extrinsic parameters are parameters of the stereo setup that give us the transformation between the two cameras. This is basically to understand where one camera is with respect to another in the 3D world.

    Several applications allow us to estimate these parameters, usually after running an experiment with a checkboard (whose dimensions are well known in advance). MATLAB's [Stereo Camera Calibrator App](https://in.mathworks.com/help/vision/ug/stereo-camera-calibrator-app.html) allows identification and exporting of the camera parameters. 
    
    Usually, once these parameters are identified, the stereo setup is frozen, that is, no changes to the camera or their relative pose is made. This is done so that we do not have to run this step every time; just run once, calibrate and done.

2. Run **stereo rectification** and **feature extraction**: This is two steps in actual, but squashed into one for describing purposes. The end result is to have features identified that belong to both the images. Only when we have correctly identified these features, can they be projected and the relevant depth information be retrieved. To understand why distinguishable features are needed consider this though experiment. Say we have two pictures taken of a plane wall (no features); what information can be projected? There is nothing that can be extracted from the plane image.

    1. Feature extraction: Finding features of significance. Usually corner detection algorithms work well. Algorithms like [SIFT](https://en.wikipedia.org/wiki/Scale-invariant_feature_transform) are also commonly used.
    2. Stereo Rectification: This means horizontally aligning the images, so that feature matching stage becomes less computationally intensive. We don't just have to find features in the two images, we also have to match them (find their corresponding equivalent in the other image).

    Usually, this can be done on the images after having the camera parameters. This is basically removing the distortion (using intrinsic parameters) from the images and horizontally aligning the two images so that individual features on each line can be matched using a linear search approach (which is less computationally intensive than having to consider pairwise features and neighborhoods).

3. Generate a **disparity map**: This basically shows the individual pixel displacement. Pixels closer to the camera have high disparity (greater deviation in the two images) than pixels that are farther from the camera in the scene. This map is useful in estimating / converting the two images into a scene description format like point cloud. Note that this can only be done on rectified images, as horizontal displacement of features are used. This is usually where the common steps end. The following steps are specific to the kind of data we want to retrieve. Usually, [Epipolar geometry](https://en.wikipedia.org/wiki/Epipolar_geometry) is heavily used in this stage and the next stages.

4. Use the disparity map and camera stereo parameters to construct the point cloud. The disparity map has already given some estimate of depth, we use the camera parameters to get the exact location of these features in the scene. Some approaches can even use apply neighborhood methods to the regions near the features in the images and virtually project every pixel into the point cloud.

This is how you could _reconstruct_ a scene as point cloud using stereo-imaging from two cameras.

**References**

- YouTube
    - [Stereo Vision by MATLAB](https://www.youtube.com/watch?v=GpU1Vx-b3VA)
- Slides
    - [Vision Research Lab, University of Minnesota](http://vision.psych.umn.edu/users/schrater/schrater_lab/courses/CompVis07/Papers/Stereo.pdf)

### Answer 3: Voxels or SDF

- Usually, SFDs have voxel information embedded in them, as they're functions that essentially take point / coordinates and return proximity to surface (which can be later used as an estimate of occupancy for that coordinate). Voxels on the other hand, are an extension of occupancy maps in 3D. They explicitly discretize (divide) the 3D space into cubical volumes and store occupancy values.
- When it comes to **preserving surface details of the scene**, SDFs can do a better job because they have that embedded in them (value = 0 is the surface). Voxels on the other hand have to run some surface detection algorithm (usually done using surface normals). This compute load is over the very heavy memory load used by Voxels. Usually, SDFs can do such tasks better than Voxel representations that use lesser memory like Octrees (which still require some processing to get surface contours of objects). Therefore, given the same memory requirement and the objective being preserving the scene objects, SDFs can do a better job.
- SDFs can be used to estimate Voxel occupancy too. This makes them a robust tool for navigation purposes. If sliced along a surface, they can be used as potential functions (that allow us to traverse a path while avoiding or repelling from obstacles).

### Answer 4: SDF Vs. Point Clouds

We might want to use SDFs when estimating occupancy in 3D space

- A point cloud has no proximity information associated with it. It is simply a collection of 3D points. To infer anything from them, some transformations need to be applied to them. To retrieve occupancy of a voxel (a cube in 3D space), points within it have to be estimated, which requires repetitive sampling of the clouds (for every query).
- SDFs on the other hand, store this information within them. For any given point in space, they return the distance from the nearest surface. This can be used as a measurement to proximity and an estimate of occupancy.

We might want to use SDFs when the objects in the scene are very important

- Since SDFs are functions (yielding the surface of obstacles / objects when equated to 0), they can be stored while containing high details about objects in the scene.
- SDFs also allow us to dilate (or contract) objects. We can simply offset the function to re-scale the object, and such scaling is very uniform. An example of this can be found [here](https://www.cineversity.com/vidplaylist/372083_default_playlist/volumetric_workflow_what_are_signed_distance_fields_sdf). Doing the same using point clouds is difficult (you have to estimate the center of scaling first, then apply transformation).

# References and Resources

1. Gimbal locks and quaternions: https://youtu.be/YF5ZUlKxSgE
2. Exponential map: 
    1. 3 Blue 1 Brown: https://youtu.be/O85OWBJ2ayo
    2. Northwestern Robotics: https://youtu.be/v_KBHaG0mas
3. Bunny ply is taken from: http://graphics.im.ntu.edu.tw/~robin/courses/cg03/model/